<a href="https://colab.research.google.com/github/2303A52393/T-268/blob/main/Brain_tumor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required libraries
!pip install torch torchvision
!pip install scikit-learn
!pip install opencv-python
!pip install kagglehub
!pip install matplotlib
!pip install seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import cv2
import kagglehub
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Download dataset
path = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")
data_path = path
train_folder = os.path.join(data_path, 'Training')
test_folder = os.path.join(data_path, 'Testing')

# Categories
categories = ['glioma', 'meningioma', 'notumor', 'pituitary']

# Custom Dataset
class BrainTumorDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Image loading and preprocessing
def load_images_from_folder(folder_path, categories, img_size):
    images, labels = [], []
    for category in categories:
        category_path = os.path.join(folder_path, category)
        if not os.path.exists(category_path):
            print(f"Folder not found: {category_path}")
            continue

        for img_name in os.listdir(category_path):
            try:
                img_path = os.path.join(category_path, img_name)
                img = cv2.imread(img_path, cv2.IMREAD_COLOR)

                if img is None:
                    print(f"Could not read image {img_path}")
                    continue

                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
                img = cv2.resize(img, (224, 224))  # Resize images

                images.append(img)
                labels.append(category)
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")

    return np.array(images), np.array(labels)

# Load images
X, y = load_images_from_folder(train_folder, categories, img_size=224)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Label encoding
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

# Transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create datasets and dataloaders
train_dataset = BrainTumorDataset(X_train, y_train_encoded, transform)
val_dataset = BrainTumorDataset(X_val, y_val_encoded, transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Simple CNN Model for Brain Tumor Classification
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)

        # Pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        # Fully connected layer
        self.fc1 = nn.Linear(128 * 28 * 28, 512)  # Assuming input image size is 224x224
        self.fc2 = nn.Linear(512, num_classes)

        # Dropout layer
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # Apply convolutional layers with ReLU activation and pooling
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))

        # Flatten the output for the fully connected layer
        x = x.view(-1, 128 * 28 * 28)  # Flattening for fully connected layers

        # Fully connected layers with dropout
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x

# Training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN(len(categories)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop with visualization
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    best_val_accuracy = 0.0

    for epoch in range(num_epochs):
        model.train()
        train_loss, train_correct = 0.0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_correct += (outputs.argmax(1) == labels).sum().item()

        # Validation
        model.eval()
        val_loss, val_correct = 0.0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                val_correct += (outputs.argmax(1) == labels).sum().item()

        train_accuracy = train_correct / len(train_loader.dataset)
        val_accuracy = val_correct / len(val_loader.dataset)

        train_losses.append(train_loss/len(train_loader))
        val_losses.append(val_loss/len(val_loader))
        train_accuracies.append(train_accuracy)
        val_accuracies.append(val_accuracy)

        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Train Loss: {train_losses[-1]:.4f}, Train Accuracy: {train_accuracy:.4f}")
        print(f"Val Loss: {val_losses[-1]:.4f}, Val Accuracy: {val_accuracy:.4f}")

        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), 'best_cnn_model.pth')

    # Plotting
    plt.figure(figsize=(12, 5))

    # Loss plot
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title('Loss Progression')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Accuracy plot
    plt.subplot(1, 2, 2)
    plt.plot(train_accuracies, label='Train Accuracy')
    plt.plot(val_accuracies, label='Validation Accuracy')
    plt.title('Accuracy Progression')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.savefig('cnn_model_performance.png')
    plt.close()

    return model, train_accuracies, val_accuracies

# Modify the existing code to use this updated training function
trained_model, train_accuracies, val_accuracies = train_model(model, train_loader, val_loader, criterion, optimizer)

# Confusion matrix visualization
def plot_confusion_matrix(model, loader, device, categories):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    from sklearn.metrics import confusion_matrix

    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=categories, yticklabels=categories)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.tight_layout()
    plt.savefig('confusion_matrix_cnn.png')
    plt.close()

# Load the best model
best_model = SimpleCNN(len(categories)).to(device)
best_model.load_state_dict(torch.load('best_cnn_model.pth'))

# Visualize confusion matrix
plot_confusion_matrix(best_model, val_loader, device, categories)

print("Performance visualizations saved as 'cnn_model_performance.png' and 'confusion_matrix_cnn.png'")

# Optional: Print final accuracies
print("\nFinal Training Accuracies:", train_accuracies)
print("Final Validation Accuracies:", val_accuracies)


Epoch 1/10
Train Loss: 0.9456, Train Accuracy: 0.6861
Val Loss: 0.4984, Val Accuracy: 0.8058
Epoch 2/10
Train Loss: 0.4713, Train Accuracy: 0.8201
Val Loss: 0.3333, Val Accuracy: 0.8775
Epoch 3/10
Train Loss: 0.3173, Train Accuracy: 0.8801
Val Loss: 0.2713, Val Accuracy: 0.8994
Epoch 4/10
Train Loss: 0.2268, Train Accuracy: 0.9114
Val Loss: 0.3090, Val Accuracy: 0.8871
Epoch 5/10
Train Loss: 0.1585, Train Accuracy: 0.9429
Val Loss: 0.2210, Val Accuracy: 0.9230
Epoch 6/10
Train Loss: 0.1235, Train Accuracy: 0.9567
Val Loss: 0.2241, Val Accuracy: 0.9300
Epoch 7/10
Train Loss: 0.0882, Train Accuracy: 0.9698
Val Loss: 0.2088, Val Accuracy: 0.9283
Epoch 8/10
Train Loss: 0.0620, Train Accuracy: 0.9810
Val Loss: 0.2509, Val Accuracy: 0.9414
Epoch 9/10
Train Loss: 0.0496, Train Accuracy: 0.9834
Val Loss: 0.2459, Val Accuracy: 0.9405
Epoch 10/10
Train Loss: 0.0468, Train Accuracy: 0.9825
Val Loss: 0.2794, Val Accuracy: 0.9379
Performance visualizations saved as 'cnn_model_performance.png' and '

In [3]:
# Calculate average accuracies
avg_train_accuracy = np.mean(train_accuracies)
avg_val_accuracy = np.mean(val_accuracies)

print(f"Average Training Accuracy: {avg_train_accuracy:.4f}")
print(f"Average Validation Accuracy: {avg_val_accuracy:.4f}")

Average Training Accuracy: 0.9114
Average Validation Accuracy: 0.9071


In [14]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, 128)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        # x shape: [batch, channels, height, width]
        x = x.permute(0, 2, 3, 1)  # [batch, height, width, channels]
        x = x.reshape(x.size(0), x.size(1), -1)  # [batch, height (seq_len), width*channels (input_size)]

        out, _ = self.lstm(x)  # [batch, seq_len, hidden]
        out = out[:, -1, :]  # Take last time step
        out = torch.relu(self.fc1(out))
        out = self.dropout(out)
        out = self.fc2(out)
        return out


In [15]:
# Model configuration
input_size = 224 * 3   # width * channels
hidden_size = 256
num_layers = 2
num_classes = len(categories)

# Instantiate LSTM model
model = LSTMClassifier(input_size, hidden_size, num_layers, num_classes).to(device)

# Training setup (same as CNN)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
trained_model, train_accuracies, val_accuracies = train_model(model, train_loader, val_loader, criterion, optimizer)

# Save best model and show confusion matrix (same code)

Epoch 1/10
Train Loss: 1.2162, Train Accuracy: 0.4152
Val Loss: 1.0963, Val Accuracy: 0.4584
Epoch 2/10
Train Loss: 1.0857, Train Accuracy: 0.4973
Val Loss: 0.9497, Val Accuracy: 0.6133
Epoch 3/10
Train Loss: 1.0881, Train Accuracy: 0.5008
Val Loss: 1.0609, Val Accuracy: 0.4794
Epoch 4/10
Train Loss: 0.9386, Train Accuracy: 0.6100
Val Loss: 0.8147, Val Accuracy: 0.6763
Epoch 5/10
Train Loss: 0.8841, Train Accuracy: 0.6463
Val Loss: 0.9377, Val Accuracy: 0.6588
Epoch 6/10
Train Loss: 0.7884, Train Accuracy: 0.6914
Val Loss: 0.7613, Val Accuracy: 0.6877
Epoch 7/10
Train Loss: 0.7324, Train Accuracy: 0.7190
Val Loss: 0.7155, Val Accuracy: 0.7314
Epoch 8/10
Train Loss: 0.7002, Train Accuracy: 0.7260
Val Loss: 0.6819, Val Accuracy: 0.7384
Epoch 9/10
Train Loss: 0.6813, Train Accuracy: 0.7446
Val Loss: 0.7556, Val Accuracy: 0.7270
Epoch 10/10
Train Loss: 0.6336, Train Accuracy: 0.7595
Val Loss: 0.7058, Val Accuracy: 0.7358
